In [12]:
import polars as pl

In [13]:
locale = 'en'
jurisdiction = 'todofuken' if locale == 'en' else 'prefecture'

In [14]:
postal_code_cats = [2, 3, 5]

In [15]:
df = pl.read_csv("data/jurisdictions.csv", dtypes={'code': str})
for n in postal_code_cats:
    df = df.with_columns(df['code'].map_elements(lambda x: x[:n]).alias(f"zip{n}"))

df.head()

code,prefecture,todofuken,zip2,zip3,zip5
str,str,str,str,str,str
"""0600000""","""北海道""","""Hokkaidou""","""06""","""060""","""06000"""
"""0640941""","""北海道""","""Hokkaidou""","""06""","""064""","""06409"""
"""0600041""","""北海道""","""Hokkaidou""","""06""","""060""","""06000"""
"""0600042""","""北海道""","""Hokkaidou""","""06""","""060""","""06000"""
"""0640820""","""北海道""","""Hokkaidou""","""06""","""064""","""06408"""


In [16]:
df.group_by(jurisdiction).agg(pl.col("code").count().alias("code_count"))

todofuken,code_count
str,u32
"""Kouchi ken""",1839
"""Tokyo to""",8730
"""Tokushima ken""",1140
"""Saga ken""",988
"""Fukui ken""",2415
"""Fukushima ken""",3961
"""Okayama ken""",2569
"""Toyama ken""",3327
"""Akita ken""",2298


In [20]:
df_list = []

for n in postal_code_cats:
    cat = f"zip{n}"
    df_cat = df.select([cat, jurisdiction])
    df_cat = df_cat.unique(maintain_order=True)
    df_cat.group_by('todofuken').agg(pl.col(cat).count().alias(f"{cat}_count"))
    df_list.append(df_cat)

In [21]:
df_list[0].head()

zip2,todofuken
str,str
"""06""","""Hokkaidou"""
"""00""","""Hokkaidou"""
"""04""","""Hokkaidou"""
"""07""","""Hokkaidou"""
"""05""","""Hokkaidou"""


In [22]:
df_list[1].head()

zip3,todofuken
str,str
"""060""","""Hokkaidou"""
"""064""","""Hokkaidou"""
"""001""","""Hokkaidou"""
"""002""","""Hokkaidou"""
"""065""","""Hokkaidou"""


In [23]:
df_list[2].head()

zip5,todofuken
str,str
"""06000""","""Hokkaidou"""
"""06409""","""Hokkaidou"""
"""06408""","""Hokkaidou"""
"""00100""","""Hokkaidou"""
"""00280""","""Hokkaidou"""
